# Simple Usage

## Basic Principles
The purpose of Maybrain is to allow easy visualisation of brain connectome and related data and perform various analyses.

The code is built around two principal classes, **brainObj** and **plotObj**. The first contains all the information about the brain and numerous functions to change, measure and highlight those. At its heart is a Networkx object, brainObj.G, via which all *Networkx* functions are available. Some have also been incorporated directly into our code for convenience. The **plotObj** class is a wrapper to enable easy plotting using Mayavi2. It can plot several brain objects and subsets of them (highlights) simultaneously and change basic plot properties. Plots are shown in the mayavi GUI for easy access to a whole range of other Mayavi functions.

## Data Input
Several types of data can be input. The basic connectome is made up of two files: a coordinate file and an adjacency matrix. In fact only the second of these is strictly required.

The coordinate file defines the position of each node. It is a text file where each line has four entries: the node index, x, y and z coordinates. e.g.:

```
0 0.0 3.1 4.4
1 5.3 7.6 8.4
2 3.2 4.4 3.1
```

The adjacency matrix defines the strength of connection between each pair of nodes. For n nodes it is an n × n text matrix. Nodes in maybrain are labelled 0,1,... and the order of the rows and columns in the adjacency matrix is assumed to correspond to this. Entering an adjacency matrix with the wrong dimensions will lead to certain doom.

## Importing Maybrain
The main functions of maybrain are contained in the brainObj module and can easily be achieved with a normal import.

In [1]:
from maybrain import brainObj as mbt

## Importing an Adjacency Matrix
Firstly, create a brain object:

In [2]:
a = mbt.brainObj()
print("Nodes: ", a.G.nodes())
print("Edges: ", a.G.edges())
print("Adjacency matrix: ", a.adjMat)

Nodes:  []
Edges:  []
Adjacency matrix:  None


This creates a brain object, where a graph is stored as `a.G`, initially empty. 

Then import the adjacency matrix (note that we are using [this file](https://github.com/RittmanResearch/maybrain/blob/master/docs/data/mean_500.txt), thus we need to add information about the delimiter). The *importAdjFile()* function imports the adjacency matrix to form the nodes of your graph, but does not create any edges (connections), as you can check from the outputs:

In [3]:
a.importAdjFile("data/mean_500.txt", delimiter=",")

print("Number of nodes:\n", a.G.number_of_nodes())
print("First 5 nodes (notice labelling starting with 0):\n", a.G.nodes()[0:5])
print("Edges:\n", a.G.edges())
print("Size of Adjacency matrix:\n", a.adjMat.shape)

Number of nodes:
 500
First 5 nodes (notice labelling starting with 0):
 [0, 1, 2, 3, 4]
Edges:
 []
Size of Adjacency matrix:
 (500, 500)


If you wish to create a fully connected graph with all the available values in the adjacency matrix, it is necessary to threshold it, which is explained in the next section.

## Thresholding
There are a few ways to apply a threshold, either using an absolute threshold across the whole graph to preserve a specified number of edges or percentage of total possible edges; or to apply a local thresholding that begins with the minimum spanning tree and adds successive n-nearest neighbour graphs. The advantage of local thresholding is that the graph will always be fully connected, which is necessary to collect some graph measures.

For an absolute threshold you have several possibilities. Note that our adjacency matrix (`a.adjMat`) always stays the same so we can apply all the thresholds we want to create our graph (`a.G`) accordingly. Also notice that we are dealing with a symmetric adjacency matrix, so although out `a.adjMat` will always have the size of 500x500, the `a.G` will not.

In [4]:
# Bring everything from the adjacency matrix to a.G
a.applyThreshold()
print("Number of edges (notice it corresponds to the upper half edges of adjacency matrix):\n", a.G.number_of_edges())
print("Size of Adjacency matrix after 1st threshold:\n", a.adjMat.shape)

# Retain the most strongly connected 1000 edges
a.applyThreshold(thresholdType = "totalEdges", value = 1000) 
print("\nNumber of edges after 2nd threshold:\n", a.G.number_of_edges())
print("Size of Adjacency matrix after 2nd threshold:\n", a.adjMat.shape)

# Retain the 5% most connected edges as a percentage of the total possible number of edges
a.applyThreshold(thresholdType = "edgePC", value = 5) 
print("\nNumber of edges after 3rd threshold:\n", a.G.number_of_edges())
print("Size of Adjacency matrix after 3rd threshold:\n", a.adjMat.shape)

# Retain edges with a weight greater than 0.3
a.applyThreshold(thresholdType = "tVal", value = 0.3) 
print("\nNumber of edges after 4th threshold:\n", a.G.number_of_edges())
print("Size of Adjacency matrix after 4th threshold:\n", a.adjMat.shape)


Number of edges (notice it corresponds to the upper half edges of adjacency matrix):
 124750
Size of Adjacency matrix after 1st threshold:
 (500, 500)

Number of edges after 2nd threshold:
 1000
Size of Adjacency matrix after 2nd threshold:
 (500, 500)

Number of edges after 3rd threshold:
 6237
Size of Adjacency matrix after 3rd threshold:
 (500, 500)

Number of edges after 4th threshold:
 34848
Size of Adjacency matrix after 4th threshold:
 (500, 500)


The options for local thresholding are similar. Note that a local thresholding always yield a connected graph, and in the case where no arguments are passed, the graph will be the Minimum Spanning Tree. Local thresholding can be very slow for bigger matrices because in each step it is adding successive N-nearest neighbour degree graphs.

In [5]:
a.localThresholding()
print("Is the graph connected? ", mbt.nx.is_connected(a.G))

a.localThresholding(thresholdType="edgePC", value = 5)
print("Is the graph connected? ", mbt.nx.is_connected(a.G))

a.localThresholding(thresholdType="totalEdges", value = 10000)
print("Is the graph connected? ", mbt.nx.is_connected(a.G))

Is the graph connected?  True
Is the graph connected?  True
Is the graph connected?  True


## Binary Graphs
If necessary the graph can be binarised so that weights are removed. You can see that essentially this means that each edge will have a weight of 1.

In [6]:
a.binarise()
print("Do all the edges have weight of 1?", all(e[2]['weight'] >= 0.5 for e in a.G.edges(data=True)))

Do all the edges have weight of 1? True


## Importing 3D Spatial Information

You can add spatial info to each node of your graph. You need this information if you want to use the visualisation tools of Maybrain.

To do so, provide Maybrain with a file that has 4 columns: an anatomical label, and x, y and z coordinates. e.g.:
```
0 70.800000 30.600000 53.320000
1 32.064909 62.154158 69.707911
2 59.870968 92.230014 41.552595
3 19.703504 66.398922 52.878706
```

Ideally these values would be in MNI space (this makes it easier to import background images for plotting and for some other functions), but this is not absolutely necessary.

We are using [this file](https://github.com/RittmanResearch/maybrain/blob/master/docs/data/parcel_500.txt) in these examples:

In [7]:
# Initially, you don't have anatomical/spatial attributes in each node:
print("Attributes: ", mbt.nx.get_node_attributes(a.G, "anatlabel"), "/", mbt.nx.get_node_attributes(a.G, "xyz"))

#After calling importSpatialInfo(), you can see the node's attributes
a.importSpatialInfo("data/parcel_500.txt")
print("Attributes of one node: ", 
      mbt.nx.get_node_attributes(a.G, "anatlabel")[0], 
      "/", 
      mbt.nx.get_node_attributes(a.G, "xyz")[0])

Attributes:  {} / {}
Attributes of one node:  0 / (70.8, 30.6, 53.32)
